<a href="https://colab.research.google.com/github/Arshiya-Begum30/Chat-with-RAG/blob/main/RAG_Model_for_QA_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.5/245.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pinecone
from groq import Groq

In [ ]:
# Replace with your actual API keys
GROQ_API_KEY="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"
PINECONE_API_KEY="150f3e68-2780-4930-a79c-4da2def2f052"

In [ ]:
# Initialize Pinecone client
pc = pinecone.Pinecone(api_key=api_key_pinecone)
index_name = "qa-bot-index"


# Create the index if it does not exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Ensure this matches your embedding dimension
        metric="euclidean",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [ ]:
# Access the index
index = pc.Index(index_name)

# Upsert vectors into the index
index.upsert(
    vectors=[
        {"id": "vec1", "values": [0.1]*1536, "metadata": {"text": "Document 1: Information about topic A."}},
        {"id": "vec2", "values": [0.2]*1536, "metadata": {"text": "Document 2: Details on subject B."}},
        {"id": "vec3", "values": [0.3]*1536, "metadata": {"text": "Document 3: Insights on topic A."}},
        {"id": "vec4", "values": [0.4]*1536, "metadata": {"text": "Document 4: Analysis of subject C."}}
    ],
    namespace="ns1"
)

{'upserted_count': 4}

In [ ]:
# Initialize Groq client
groq_client = Groq(api_key=api_key_groq)

# Function to encode queries to vectors (Placeholder implementation)
def encode_query_to_vector(query):
    # Implement your vector encoding logic here
    # This is a placeholder that should be replaced with actual encoding
    return [0.3]*1536

In [ ]:
# Define a function to retrieve relevant documents from Pinecone
def retrieve_documents(query, top_k=5):
    query_vector = encode_query_to_vector(query)  # Implement the actual encoding function
    results = index.query(
        namespace="ns1",
        vector=query_vector,
        top_k=top_k,
        include_values=True,
        include_metadata=True
    )
    retrieved_docs = [match['metadata']['text'] for match in results['matches']]
    return retrieved_docs

In [ ]:
# Define a function to perform RAG-based chat completion using Groq
def get_rag_chat_completion(prompt):
    try:
        # Retrieve relevant documents based on the prompt
        retrieved_docs = retrieve_documents(prompt)

        # Combine the prompt with the retrieved documents
        combined_prompt = prompt + "\n\n" + "\n".join(retrieved_docs)

        # Perform a chat completion with Groq
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": combined_prompt
                }
            ],
            model="llama3-8b-8192"  # Replace with the model you want to use
        )
        # Return the response content
        return chat_completion.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
if __name__ == "__main__":
    prompt = "Explain the significance of topic A."
    response = get_rag_chat_completion(prompt)
    print("Response from Groq:", response)

Response from Groq: A clever arrangement!

It seems that "topic A" is a recurring theme throughout these documents. Here's a possible interpretation:

* Document 1: This is a foundational text that provides an overview of topic A, introducing the reader to its significance and importance.
* Document 2: This text provides more details about a related subject B, which might be a subtopic or a specific aspect of topic A.
* Document 3: This document delves deeper into topic A, offering insights and analysis that build upon the information provided in Document 1.
* Document 4: This text examines a separate subject C, which might be connected to topic A or provide a contrasting perspective.

The significance of topic A lies in its recurring presence across these documents. It likely represents a core concept, idea, or theme that is central to the overall narrative or discussion. By focusing on topic A, the reader can gain a deeper understanding of the larger context and the connections betwe